# Нарезка города на кварталы

In [ ]:
!pip install blocksnet==0.0.12

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 98.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.8/277.8 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 100.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.1/255.1 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 105.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.9/38.9 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 104.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 64.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━

In [ ]:
import osmnx as ox

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
boundary = ox.geocode_to_gdf('г. Кинешма') #Можно, например заменить на Sankt-Peterburg

In [ ]:
tags = {
    'roads': {
      "highway": ["construction","crossing","living_street","motorway","motorway_link","motorway_junction","pedestrian","primary","primary_link","raceway","residential","road","secondary","secondary_link","services","tertiary","tertiary_link","trunk","trunk_link","turning_circle","turning_loop","unclassified",],
      "service": ["living_street", "emergency_access"]
    },
    'railways': {
      "railway": "rail"
    },
    'water': {
      'riverbank':True,
      'reservoir':True,
      'basin':True,
      'dock':True,
      'canal':True,
      'pond':True,
      'natural':['water','bay'],
      'waterway':['river','canal','ditch'],
      'landuse':'basin',
      'water': 'lake'
    }
}

In [ ]:
water = ox.features_from_polygon(boundary.unary_union, tags['water'])
roads = ox.features_from_polygon(boundary.unary_union, tags['roads'])
railways = ox.features_from_polygon(boundary.unary_union, tags['railways'])

Определяем локальную CRS для города в метрах, к ней будем приводить все имеющиеся геометрии (и, соответственно, получать слой кварталов).

In [ ]:
local_crs = boundary.estimate_utm_crs()

In [ ]:
boundary = boundary.reset_index()[['geometry']].to_crs(local_crs)
water = water.reset_index()[['geometry']].to_crs(local_crs)
roads = roads.reset_index()[['geometry']].to_crs(local_crs)
railways = railways.reset_index()[['geometry']].to_crs(local_crs)

In [ ]:
roads = roads[roads.geom_type.isin(['LineString', 'MultiLineString'])]

In [ ]:
roads

,geometry
41,"LINESTRING (324143.860 6373570.152, 324152.988..."
42,"LINESTRING (324207.650 6373768.422, 324200.892..."
43,"LINESTRING (326388.836 6372192.968, 326421.535..."
44,"LINESTRING (330661.706 6370031.041, 330559.809..."
45,"LINESTRING (327962.448 6369950.428, 327823.986..."
...,...
885,"LINESTRING (325314.313 6369455.724, 325264.892..."
886,"LINESTRING (326079.558 6371552.467, 326042.900..."
887,"LINESTRING (326144.017 6371338.566, 326135.443..."
888,"LINESTRING (326417.464 6371814.749, 326497.411..."


Импортируем класс `BlocksGenerator` и инициализируем его экземпляр с помощью наших геометрий. На этапе инициализации:
- все геометрии проверяются по спецификации (можете увидеть при наведении курсора на BlocksGenerator);
- из геометрии границ территории удаляются полигоны водных объектов.

In [ ]:
from blocksnet import BlocksGenerator

bg = BlocksGenerator(boundary, roads, railways, water)

2025-04-24 02:40:01.822 | INFO     | blocksnet.preprocessing.blocks_generator:__init__:99 - Check boundaries schema
2025-04-24 02:40:01.830 | INFO     | blocksnet.preprocessing.blocks_generator:__init__:103 - Check roads schema
2025-04-24 02:40:01.839 | INFO     | blocksnet.preprocessing.blocks_generator:__init__:109 - Check railways schema
2025-04-24 02:40:01.846 | INFO     | blocksnet.preprocessing.blocks_generator:__init__:115 - Check water schema
2025-04-24 02:40:01.853 | INFO     | blocksnet.preprocessing.blocks_generator:__init__:124 - Exclude water objects


In [ ]:
blocks = bg.run()

2025-04-24 02:40:02.952 | INFO     | blocksnet.preprocessing.blocks_generator:run:161 - Generating blocks
2025-04-24 02:40:02.965 | INFO     | blocksnet.preprocessing.blocks_generator:run:170 - Setting up enclosures
2025-04-24 02:40:03.106 | INFO     | blocksnet.preprocessing.blocks_generator:run:174 - Filling holes
2025-04-24 02:40:03.167 | INFO     | blocksnet.preprocessing.blocks_generator:run:178 - Dropping overlapping blocks
2025-04-24 02:40:03.348 | INFO     | blocksnet.preprocessing.blocks_generator:run:192 - Calculating blocks area
2025-04-24 02:40:03.363 | INFO     | blocksnet.preprocessing.blocks_generator:run:199 - Blocks generated


С помощью библиотеки mapclassify мы можем выводить геометрии на карту через `.explore()`.

In [ ]:
!pip install mapclassify -q

In [ ]:
blocks.explore()

По желанию можно дорезать кварталы с помощью зданий. Не забываем временно перевести в EPSG:4326, ведь мы уже меняли CRS раньше. От них нам достаточно геометрий, поэтому забираем все.

In [ ]:
buildings = ox.features_from_polygon(boundary.to_crs(4326).unary_union, {'building': True})

In [ ]:
buildings

building  \
element_type osmid                 
node         3783805078  garages   
way          68168007        yes   
             68168009        yes   
             68168012        yes   
             68168014        yes   
...                          ...   
relation     14772839        yes   
             14772840        yes   
             14772841        yes   
             14772842        yes   
             14773015        yes   

                                                                  geometry  \
element_type osmid                                                           
node         3783805078                          POINT (42.11444 57.44082)   
way          68168007    POLYGON ((42.09373 57.44698, 42.09407 57.44707...   
             68168009    POLYGON ((42.13690 57.44881, 42.13683 57.44892...   
             68168012    POLYGON ((42.09219 57.45169, 42.09235 57.45177...   
             68168014    POLYGON ((42.09284 57.45200, 42.09300 57.45208...   
...                                                                    ...   
relation     14772839    MULTIPOLYGON (((42.10079 57.45939, 42.10066 57...   
             14772840    MULTIPOLYGON (((42.09981 57.45682, 42.10065 57...   
             14772841    MULTIPOLYGON (((42.10087 57.45762, 42.10052 57...   
             14772842    MULTIPOLYGON (((42.09900 57.45716, 42.09886 57...   
             14773015    MULTIPOLYGON (((42.10557 57.46159, 42.10514 57...   

                        amenity brand contact:facebook contact:phone  \
element_type osmid                                                     
node         3783805078     NaN   NaN              NaN           NaN   
way          68168007       NaN   NaN              NaN           NaN   
             68168009       NaN   NaN              NaN           NaN   
             68168012       NaN   NaN              NaN           NaN   
             68168014       NaN   NaN              NaN           NaN   
...                         ...   ...              ...           ...   
relation     14772839       NaN   NaN              NaN           NaN   
             14772840       NaN   NaN              NaN           NaN   
             14772841       NaN   NaN              NaN           NaN   
             14772842       NaN   NaN              NaN           NaN   
             14773015       NaN   NaN              NaN           NaN   

                        contact:vk contact:website name name:en  ...  \
element_type osmid                                               ...   
node         3783805078        NaN             NaN  NaN     NaN  ...   
way          68168007          NaN             NaN  NaN     NaN  ...   
             68168009          NaN             NaN  NaN     NaN  ...   
             68168012          NaN             NaN  NaN     NaN  ...   
             68168014          NaN             NaN  NaN     NaN  ...   
...                            ...             ...  ...     ...  ...   
relation     14772839          NaN             NaN  NaN     NaN  ...   
             14772840          NaN             NaN  NaN     NaN  ...   
             14772841          NaN             NaN  NaN     NaN  ...   
             14772842          NaN             NaN  NaN     NaN  ...   
             14773015          NaN             NaN  NaN     NaN  ...   

                        building:flats power substation image denomination  \
element_type osmid                                                           
node         3783805078            NaN   NaN        NaN   NaN          NaN   
way          68168007              NaN   NaN        NaN   NaN          NaN   
             68168009              NaN   NaN        NaN   NaN          NaN   
             68168012              NaN   NaN        NaN   NaN          NaN   
             68168014              NaN   NaN        NaN   NaN          NaN   
...                                ...   ...        ...   ...          ...   
relation     14772839              NaN   N

Переводим здания в нашу CRS и оставляем только столбец с геометрияaми. От них берем центроиды (`.representative_point()` - точка центра, которая точно будет лежать на полигоне).


In [ ]:
buildings = buildings.to_crs(local_crs).reset_index()[['geometry']]
buildings.geometry = buildings.representative_point()

In [ ]:
from blocksnet import BlocksSplitter

bs = BlocksSplitter(blocks, buildings)

Здесь можно поиграть со следующими параметрами кластеризации и выбора необходимых кварталов для нарезки:
- n_clusters : int
    Number of clusters to form within each block, default is 4.

- points_quantile : float
    Quantile value to filter blocks by the number of points, default is 0.98.

- area_quantile : float
    Quantile value to filter blocks by area, default is 0.95.

In [ ]:
splitted_blocks = bs.run()

2025-04-24 02:40:27.268 | INFO     | blocksnet.preprocessing.blocks_splitter:run:167 - Joining buildings and blocks to exclude duplicates
2025-04-24 02:40:27.555 | INFO     | blocksnet.preprocessing.blocks_splitter:run:176 - Choosing blocks to be splitted
2025-04-24 02:40:27.652 | INFO     | blocksnet.preprocessing.blocks_splitter:run:185 - Splitting filtered blocks
100%|██████████| 14/14 [00:10<00:00,  1.33it/s]


Как мы видим, кварталов стало побольше примерно на 200.

In [ ]:
len(blocks), len(splitted_blocks)

(743, 791)

In [ ]:
splitted_blocks.to_file('blocks_Kineshma.geojson') #Сохранение кварталов

# Восполнение населения в зданиях




## Восполнение населения для 2010 года

In [ ]:
import geopandas as gpd
import pandas as pd

# Загрузка геослоя с полигонами зданий
buildings = gpd.read_file('buildings_2010.geojson')

# Преобразуем столбец 'building:levels' в числовой тип, ошибки будут заменены на NaN
buildings['building:levels'] = pd.to_numeric(buildings['building:levels'], errors='coerce')

# Заполним пропущенные значения (NaN) нулями или другими подходящими значениями
buildings = buildings.fillna(0)

# Добавляем или преобразуем необходимые атрибуты

# 1. Количество этажей (number_of_floors)
buildings['number_of_floors'] = buildings.apply(
    lambda x: x['building:levels'] if x['building:levels'] > 1 else 1,
    axis=1
)

# 2. Площадь застройки (footprint_area) - как площадь геометрии (основание здания)
buildings['footprint_area'] = buildings.geometry.area

# 3. Общая площадь всех этажей (build_floor_area) - footprint_area * number_of_floors
buildings['build_floor_area'] = buildings['footprint_area'] * buildings['number_of_floors']

# 4. Жилая площадь (living_area) и нежилая площадь (non_living_area)
# Жилая площадь будет рассчитываться только для жилых зданий
residential_tags = ['residential', 'house', 'apartments', 'detached', 'terrace', 'dormitory']
buildings['living_area'] = buildings.apply(
    lambda x: 0.8 * x['build_floor_area'] if x['building'] in residential_tags else 0,
    axis=1
)

# Нежилая площадь будет 20% от общей площади этажей
buildings['non_living_area'] = buildings['build_floor_area'] - buildings['living_area']

# 5. Население (population) - для жилых зданий
buildings['population'] = buildings.apply(
    lambda x: 48 * x['number_of_floors'] if x['building'] in residential_tags else 0,
    axis=1
)

# Масштабируем население, указав численность населения в городе в 2010 году
total_population = buildings['population'].sum()
scaling_factor = 77694 / total_population if total_population > 0 else 1
buildings['population'] = (buildings['population'] * scaling_factor).round()

# Теперь удалим все остальные столбцы, кроме 'geometry', 'build_floor_area', 'living_area', 'non_living_area',
# 'footprint_area', 'number_of_floors', 'population'
buildings = buildings[['geometry', 'build_floor_area', 'living_area', 'non_living_area',
                       'footprint_area', 'number_of_floors', 'population']]

# Проверим результат
print(buildings)


                                                geometry  build_floor_area  \
0      MULTIPOLYGON (((42.111 57.41512, 42.11071 57.4...      2.973231e-07   
1      MULTIPOLYGON (((42.10667 57.42959, 42.10677 57...      5.433546e-07   
2      MULTIPOLYGON (((42.19036 57.43802, 42.19006 57...      2.187857e-07   
3      MULTIPOLYGON (((42.11908 57.45295, 42.11921 57...      3.028473e-07   
4      MULTIPOLYGON (((42.11868 57.45007, 42.11883 57...      2.247275e-07   
...                                                  ...               ...   
11378  MULTIPOLYGON (((42.13428 57.45282, 42.1342 57....      1.473345e-08   
11379  MULTIPOLYGON (((42.15076 57.44355, 42.15088 57...      1.818082e-07   
11380  MULTIPOLYGON (((42.16232 57.43497, 42.16251 57...      6.926782e-08   
11381  MULTIPOLYGON (((42.0948 57.44611, 42.09515 57....      2.087134e-07   
11382  MULTIPOLYGON (((42.16873 57.44205, 42.16896 57...      3.714144e-08   

        living_area  non_living_area  footprint_area  number_of

<ipython-input-14-f7e57fc0000c>:22: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  buildings['footprint_area'] = buildings.geometry.area


In [ ]:
# Сохранение результата в новый GeoJSON файл
buildings.to_file('buildings_2010_population.geojson', driver='GeoJSON')

## Восполнение населения для 2025 года

In [ ]:
import geopandas as gpd
import pandas as pd

# Загрузка геослоя с полигонами зданий 2025
buildings = gpd.read_file('buildings_2025.geojson')

# Преобразуем столбец 'building:levels' в числовой тип, ошибки будут заменены на NaN
buildings['building:levels'] = pd.to_numeric(buildings['building:levels'], errors='coerce')

# Заполним пропущенные значения (NaN) нулями или другими подходящими значениями
buildings = buildings.fillna(0)

# Добавляем или преобразуем необходимые атрибуты

# 1. Количество этажей (number_of_floors)
buildings['number_of_floors'] = buildings.apply(
    lambda x: x['building:levels'] if x['building:levels'] > 1 else 1,
    axis=1
)

# 2. Площадь застройки (footprint_area) - как площадь геометрии (основание здания)
buildings['footprint_area'] = buildings.geometry.area

# 3. Общая площадь всех этажей (build_floor_area) - footprint_area * number_of_floors
buildings['build_floor_area'] = buildings['footprint_area'] * buildings['number_of_floors']

# 4. Жилая площадь (living_area) и нежилая площадь (non_living_area)
# Жилая площадь будет рассчитываться только для жилых зданий
residential_tags = ['residential', 'house', 'apartments', 'detached', 'terrace', 'dormitory']
buildings['living_area'] = buildings.apply(
    lambda x: 0.8 * x['build_floor_area'] if x['building'] in residential_tags else 0,
    axis=1
)

# Нежилая площадь будет 20% от общей площади этажей
buildings['non_living_area'] = buildings['build_floor_area'] - buildings['living_area']

# 5. Население (population) - для жилых зданий
buildings['population'] = buildings.apply(
    lambda x: 48 * x['number_of_floors'] if x['building'] in residential_tags else 0,
    axis=1
)

# Масштабируем население по данным 2025 года (или 2021)
total_population = buildings['population'].sum()
scaling_factor = 77694 / total_population if total_population > 0 else 1
buildings['population'] = (buildings['population'] * scaling_factor).round()

# Теперь удалим все остальные столбцы, кроме 'geometry', 'build_floor_area', 'living_area', 'non_living_area',
# 'footprint_area', 'number_of_floors', 'population'
buildings = buildings[['geometry', 'build_floor_area', 'living_area', 'non_living_area',
                       'footprint_area', 'number_of_floors', 'population']]

# Проверим результат
print(buildings)


                                                geometry  build_floor_area  \
0      MULTIPOLYGON (((42.111 57.41512, 42.11071 57.4...      2.973231e-07   
1      MULTIPOLYGON (((42.10667 57.42959, 42.10677 57...      5.433546e-07   
2      MULTIPOLYGON (((42.19036 57.43802, 42.19006 57...      2.187857e-07   
3      MULTIPOLYGON (((42.11908 57.45295, 42.11921 57...      3.028473e-07   
4      MULTIPOLYGON (((42.11868 57.45007, 42.11883 57...      2.247275e-07   
...                                                  ...               ...   
11378  MULTIPOLYGON (((42.13428 57.45282, 42.1342 57....      1.473345e-08   
11379  MULTIPOLYGON (((42.15076 57.44355, 42.15088 57...      1.818082e-07   
11380  MULTIPOLYGON (((42.16232 57.43497, 42.16251 57...      6.926782e-08   
11381  MULTIPOLYGON (((42.0948 57.44611, 42.09515 57....      2.087134e-07   
11382  MULTIPOLYGON (((42.16873 57.44205, 42.16896 57...      3.714144e-08   

        living_area  non_living_area  footprint_area  number_of

<ipython-input-14-f7e57fc0000c>:22: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  buildings['footprint_area'] = buildings.geometry.area


In [ ]:
# Сохранение результата в новый GeoJSON файл
buildings.to_file('buildings_2025_population.geojson', driver='GeoJSON')

# Оценка изменения численности населения по районам

In [ ]:
import geopandas as gpd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Загрузка границ районов
gdf = gpd.read_file("ocity_boundary.geojson")

# Вычисление изменения населения
gdf["population_change"] = gdf["population_2025"] - gdf["population_2010"]

# Нормализация симметрично от -1 до 1
change_values = gdf["population_change"].values.reshape(-1, 1)
max_abs = np.max(np.abs(change_values))
gdf["population_change_normalized"] = (change_values / max_abs).flatten()

# Загрузка кварталов
blocks = gpd.read_file("blocks.geojson")  # предполагаемое имя слоя с кварталами

# Приведение CRS, если требуется (чтобы был один и тот же)
if blocks.crs != gdf.crs:
    blocks = blocks.to_crs(gdf.crs)

# Пространственное объединение: экстраполяция значения изменения населения на кварталы
blocks_with_change = gpd.sjoin(blocks, gdf[["population_change", "population_change_normalized", "geometry"]], how="left", predicate="intersects")

# Сохраняем объединённый слой
blocks_with_change.to_file("blocks_with_population_change.geojson", driver="GeoJSON")

# Оценка изменения плотности населения по кварталам

In [ ]:
import geopandas as gpd

# Загрузка данных
blocks = gpd.read_file('blocks.geojson')  # кварталы
buildings_2012 = gpd.read_file('buildings_2010_population.geojson')  # застройка 2012
buildings_2025 = gpd.read_file('buildings_2025_population.geojson')  # застройка 2025

# === 2. Приведение к общей проекции (в метрах) ===
blocks = blocks.to_crs(epsg=3857)
buildings_2012 = buildings_2012.to_crs(blocks.crs)
buildings_2025 = buildings_2025.to_crs(blocks.crs)

# === 3. Добавление уникального ID кварталам ===
blocks['block_id'] = blocks.index

# === 4. Расчёт площади кварталов (в м²) ===
blocks['area_m2'] = blocks.geometry.area

# === 5. Пересечение зданий и кварталов ===
buildings_2012_in_blocks = gpd.overlay(buildings_2012, blocks[['block_id', 'geometry']], how='intersection')
buildings_2025_in_blocks = gpd.overlay(buildings_2025, blocks[['block_id', 'geometry']], how='intersection')

# === 6. Суммарное население в кварталах ===
pop_2012 = buildings_2012_in_blocks.groupby('block_id')['population'].sum()
pop_2025 = buildings_2025_in_blocks.groupby('block_id')['population'].sum()

# === 7. Добавляем население в кварталы ===
blocks['pop_2012'] = blocks['block_id'].map(pop_2012).fillna(0)
blocks['pop_2025'] = blocks['block_id'].map(pop_2025).fillna(0)

# === 8. Расчёт плотности населения (чел/км²) ===
blocks['density_2012'] = blocks['pop_2012'] / (blocks['area_m2'] / 1_000_000)
blocks['density_2025'] = blocks['pop_2025'] / (blocks['area_m2'] / 1_000_000)

# === 9. Разница плотности ===
blocks['density_diff'] = blocks['density_2025'] - blocks['density_2012']

# === 10. Нормализация разницы от -1 до 1 (с сохранением 0) ===
max_abs_diff = blocks['density_diff'].abs().max()
blocks['density_diff_norm'] = blocks['density_diff'] / max_abs_diff

# === 11. Сохраняем результат в GeoJSON ===
blocks[['geometry', 'density_2012', 'density_2025', 'density_diff', 'density_diff_norm']].to_file(
    'blocks_density_diff.geojson',
    driver='GeoJSON'
)

print("✅ Расчёты завершены. Файл сохранён как 'blocks_density_diff.geojson'.")

# Формирование квартально-сетевой модели города

### Подготовка

Устанавливаем библиотеки.

Подробнее ознакомиться с библиотекой для построения транспортных графов IduEdu можно на [GitHub](https://github.com/DDonnyy/IduEdu/tree/main).


In [ ]:
!pip install blocksnet iduedu mapclassify -qq

In [ ]:
import geopandas as gpd
import os

In [ ]:
data_path = '/content' # в эту папку нужно положить geojson файл с нарезанными кварталми


In [ ]:
blocks = gpd.read_file(os.path.join(data_path, '/content/blocks.geojson')) # считываем в переменную наши кварталы

### Сборка дорожного графа

In [ ]:
from iduedu import get_boundary

bounds = get_boundary(osm_id=389795) # (osm_id=...) передаем OSM id границ города, в данном примере г. Орск

Импортируем метод ```get_drive_graph``` для скачивания дорожного графа по полигону.

In [ ]:
from iduedu import get_drive_graph

G_drive = get_drive_graph(polygon=bounds, additional_edgedata=['highway', 'maxspeed', 'reg', 'ref','name'])

In [ ]:
G_drive # в этой переменной хранится собранный граф

In [ ]:
from blocksnet import AccessibilityProcessor
AccessibilityProcessor._fix_graph(G_drive) # необходимо для исправления графа, запустить!

In [ ]:
n,e = ox.graph_to_gdfs(G_drive)

```GeoDataFrame``` с вершинами дорожного графа (нодами, ```n``` – сокр. nodes).




In [ ]:
n

,y,x,street_count,highway,geometry
osmid,,,,,
322874874,6.368488e+06,325136.175400,3,NaN,POINT (325136.175 6368488.335)
3801324834,6.368644e+06,325330.691013,4,NaN,POINT (325330.691 6368644.169)
1187490993,6.369116e+06,324616.307325,4,NaN,POINT (324616.307 6369116.305)
444729378,6.373656e+06,324201.458199,3,NaN,POINT (324201.458 6373656.462)
3789665857,6.373668e+06,324224.585928,3,NaN,POINT (324224.586 6373667.870)
...,...,...,...,...,...
10021975100,6.369068e+06,324852.675233,1,NaN,POINT (324852.675 6369067.958)
10021975106,6.369131e+06,324895.395797,1,NaN,POINT (324895.396 6369131.465)
10022607925,6.371516e+06,326042.900086,1,NaN,POINT (326042.900 6371515.855)


```GeoDataFrame``` с ребрами дорожного графа (эджи, ```e``` – сокр. edges).

Колонка ```time_min``` содержит в себе значения времени в минутах, за которое можно проехать на автомобиле данный участок дороги.

In [ ]:
e

length_meter  time_min  \
u           v           key                           
322874874   3801324834  0         250.108     0.250   
            1187490993  0         827.380     0.827   
3801324834  3801324786  0         107.369     0.161   
            322874874   0         250.108     0.250   
            821355091   0         152.458     0.152   
...                                   ...       ...   
10021975100 10021975099 0          85.166     0.128   
10021975106 10021975099 0         183.205     0.275   
10022607925 10022607924 0          51.810     0.078   
10022608166 10022608204 0         128.912     0.193   
10022608203 10022608204 0         130.641     0.196   

                                                                      geometry  \
u           v           key                                                      
322874874   3801324834  0    LINESTRING (325136.175 6368488.335, 325206.606...   
            1187490993  0    LINESTRING (325136.175 6368488.335, 325128.530...   
3801324834  3801324786  0    LINESTRING (325330.691 6368644.169, 325387.205...   
            322874874   0    LINESTRING (325330.691 6368644.169, 325309.859...   
            821355091   0    LINESTRING (325330.691 6368644.169, 325355.227...   
...                                                                        ...   
10021975100 10021975099 0    LINESTRING (324852.675 6369067.958, 324796.437...   
10021975106 10021975099 0    LINESTRING (324895.396 6369131.465, 324911.676...   
10022607925 10022607924 0    LINESTRING (326042.900 6371515.855, 326079.558...   
10022608166 10022608204 0    LINESTRING (326599.621 6371789.915, 326513.699...   
10022608203 10022608204 0    LINESTRING (326417.464 6371814.749, 326497.411...   

                                 highway     maxspeed  reg      ref  \
u           v           key                                           
322874874   3801324834  0      secondary  1000.000000    2  24К-108   
            1187490993  0        primary  1000.000000    2      NaN   
3801324834  3801324786  0    residential   666.666667    3      NaN   
            322874874   0      secondary  1000.000000    2  24К-108   
            821355091   0      secondary  1000.000000    2  24К-108   
...                                  ...          ...  ...      ...   
10021975100 10021975099 0    residential   666.666667    3      NaN   
10021975106 10021975099 0    residential   666.666667    3      NaN   
10022607925 10022607924 0    residential   666.666667    3      NaN   
10022608166 10022608204 0    residential   666.666667    3      NaN   
10022608203 10022608204 0    residential   666.666667    3      NaN   

                                                                          name  
u           v           key                                                     
322874874   3801324834  0                                      Вичугская улица  
            1187490993  0                                                  Р71  
3801324834  3801324786  0                                                  NaN  
            322874874   0                                      Вичугская улица  
            821355091   0                                      Вичугская улица  
...                                                                        ...  
10021975100 10021975099 0                            улица Народного Ополчения  
10021975106 10021975099 0    [улица им. Григория Лапши, улица Фёдора Красного]  
10022607925 10022607924 0                                                  NaN  
10022608166 10022608204 0                                  6-й Почтовый проезд  
10022608203 10022608204 0                                      Почтовый проезд  

[4193 rows x 8 columns]

Сохраняем вершины и ребра графа в файлы GeoJSON.

In [ ]:
n.to_file(os.path.join(data_path,"G_drive_nodes.geojson"))

In [ ]:
e.to_file(os.path.join(data_path,"G_drive_edges.geojson"))

### Создание матрицы доступности и квартально-сетевой модели + вычисление транспортной доступности и связности

Импортируем из BlocksNet класс ```AccessibilityProcessor```, необходимый для вычисления матрицы доступности по кварталам.

Создаем экземпляр класса в переменной ```ap``` и передаем туда кварталы ```blocks```.

In [ ]:
from blocksnet import AccessibilityProcessor

ap = AccessibilityProcessor(blocks)

Вызываем метод ```get_accessibility_matrix``` у экземпляра ```ap```, передавая дорожный граф. Переменная ```acc_mx``` будет содержать матрицу доступности.

In [ ]:
acc_mx = ap.get_accessibility_matrix(G_drive)
acc_mx.head() # вывод первых 5 строк полученной матрицы

,0,1,2,3,4,5,6,7,8,9,...,781,782,783,784,785,786,787,788,789,790
0,0.000000,1.326172,2.060547,0.623047,10.101562,9.812500,9.304688,8.062500,7.007812,7.593750,...,3.953125,8.328125,6.210938,6.210938,6.210938,8.320312,5.617188,5.617188,8.406250,6.089844
1,1.326172,0.000000,2.849609,1.859375,10.890625,10.601562,10.093750,8.851562,7.796875,8.382812,...,4.742188,8.929688,7.000000,7.000000,7.000000,9.109375,6.406250,6.406250,9.195312,6.882812
2,2.060547,2.849609,0.000000,2.593750,10.820312,10.531250,10.023438,8.781250,7.726562,8.312500,...,1.892578,8.992188,6.929688,6.929688,6.929688,9.039062,6.335938,6.335938,9.125000,6.812500
3,0.623047,1.859375,2.593750,0.000000,10.632812,10.343750,9.835938,8.601562,7.539062,8.125000,...,4.484375,8.859375,6.742188,6.742188,6.742188,8.851562,6.148438,6.148438,8.937500,6.625000
4,9.945312,10.734375,10.671875,10.484375,0.000000,1.299805,1.372070,2.037109,3.119141,2.800781,...,9.804688,7.750000,4.441406,4.441406,4.441406,7.738281,5.039062,5.039062,5.753906,4.562500


Сохраняем матрицу доступности в pickle файл.

In [ ]:
acc_mx.to_pickle(os.path.join(data_path, 'acc_mx.pickle'))

Импортируем из ```blocksnet.models``` класс ```City```.

In [ ]:
from blocksnet.models import City

Для создания квартально-сетевой модели города используются кварталы и матрица доступности. Чтобы модель собралась с учетом валидации данных, которая встроена в класс ```City```, необходимо добавить в кварталы информацию о зонировании территорий (землепользование). В случае если таких данных нет, то нужно добавить дополнительную колонку ```land_use``` с значениями ```None``` в ГеоДатаФрейм с кварталми.

In [ ]:
blocks['land_use'] = None

Создаем экземпляр класса в переменную ```city``` и отдаем туда кварталы и матрицу доступности.

In [ ]:
city = City(
    blocks=blocks,
    acc_mx=acc_mx
)

Вывод основной информации о собранной модели: CRS, кол-во кварталов, типов сервисов, зданий и сервисов.

In [ ]:
print(city)

CRS : EPSG:32638
Blocks : 791
Service types : 0/66
Buildings : 0
Services : 0



Сохраняем модельку в pickle файл. Она вам пригодится на следующем практическом занятии :)

In [ ]:
city.to_pickle(os.path.join(data_path,'city_model.pickle'))

# Оценка обеспеченности

## Загрузка сервисов в модель



HOSPITAL

In [ ]:
import geopandas as gpd
import pandas as pd

# Шаг 0: Загрузка hospital вручную из файла (например, 'my_hospitals.geojson')
# Замените путь и формат файла на нужный вам
hospital = gpd.read_file("Kin_hospital.geojson")  # или .shp, .gpkg и т.п.

# Приведение CRS к локальной системе координат
hospital = hospital.to_crs(local_crs)

# Шаг 1: Удаляем вложенные полигоны
def remove_inner_polygons(gdf_polygons):
    to_remove = []
    for idx, poly1 in gdf_polygons.iterrows():
        for idx2, poly2 in gdf_polygons.iterrows():
            if idx != idx2 and poly1.geometry.contains(poly2.geometry):
                to_remove.append(idx2)
    return gdf_polygons.drop(to_remove)

hospital = remove_inner_polygons(hospital)

# Шаг 2: Преобразуем полигоны в центроиды
hospital['geometry'] = hospital.centroid

# Шаг 3: Удаляем близкие центроиды, оставляя одну точку на буфер
def remove_close_centroids(gdf_centroids, buffer_distance=60):
    final_centroids = gpd.GeoDataFrame(columns=gdf_centroids.columns, crs=gdf_centroids.crs)

    while not gdf_centroids.empty:
        current_point = gdf_centroids.iloc[0]
        buffer = current_point.geometry.buffer(buffer_distance)
        close_points = gdf_centroids[gdf_centroids.geometry.within(buffer)]

        final_centroids = gpd.GeoDataFrame(
            pd.concat([final_centroids, gpd.GeoDataFrame([current_point])], ignore_index=True),
            crs=gdf_centroids.crs
        )

        gdf_centroids = gdf_centroids.drop(close_points.index)

    return final_centroids

hospital = remove_close_centroids(hospital)

hospital.explore()

# Оставляем только геометрию
hospital_geometry = hospital[['geometry']]

# Обновляем данные в модели города
city.update_services('hospital', hospital_geometry)

services_gdf = city.get_services_gdf()
services_gdf.head()


/usr/local/lib/python3.11/dist-packages/geopandas/array.py:1470: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 / UTM zone 38N (the single non-null crs provided).
  return GeometryArray(data, crs=_get_common_crs(to_concat))
/usr/local/lib/python3.11/dist-packages/geopandas/array.py:1470: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 / UTM zone 38N (the single non-null crs provided).
  return GeometryArray(data, crs=_get_common_crs(to_concat))
/usr/local/lib/python3.11/dist-packages/geopandas/array.py:1470: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 / UTM zone 38N (the single non-null crs provided).
  return GeometryArray(data, crs=_get_common_crs(to_concat))


,geometry,block_id,building_id,service_type,capacity,area,is_integrated
0,POINT (326597.822 6370355.923),18,734.0,hospital,110,3500.0,False
1,POINT (329897.131 6370417.726),380,166.0,hospital,110,3500.0,False
2,POINT (333046.966 6369149.858),566,NaN,hospital,110,3500.0,False


SCHOOL

In [ ]:
import geopandas as gpd
import pandas as pd

# Шаг 0: Загрузка school вручную из файла (например, 'my_schools.geojson')
# Замените путь и формат файла на нужный вам
school = gpd.read_file("Kin_school_2010_centroids.geojson")  # или .shp, .gpkg, .csv и т.п.

# Приведение CRS к локальной системе координат
school = school.to_crs(local_crs)

# Шаг 1: Удаляем вложенные полигоны
def remove_inner_polygons(gdf_polygons):
    to_remove = []
    for idx, poly1 in gdf_polygons.iterrows():
        for idx2, poly2 in gdf_polygons.iterrows():
            if idx != idx2 and poly1.geometry.contains(poly2.geometry):
                to_remove.append(idx2)
    return gdf_polygons.drop(to_remove)

school = remove_inner_polygons(school)

# Шаг 2: Преобразуем полигоны в центроиды
school['geometry'] = school.centroid

# Шаг 3: Удаляем близкие центроиды, оставляя одну точку на буфер
def remove_close_centroids(gdf_centroids, buffer_distance=60):
    final_centroids = gpd.GeoDataFrame(columns=gdf_centroids.columns, crs=gdf_centroids.crs)

    while not gdf_centroids.empty:
        current_point = gdf_centroids.iloc[0]
        buffer = current_point.geometry.buffer(buffer_distance)
        close_points = gdf_centroids[gdf_centroids.geometry.within(buffer)]

        final_centroids = gpd.GeoDataFrame(
            pd.concat([final_centroids, gpd.GeoDataFrame([current_point])], ignore_index=True),
            crs=gdf_centroids.crs
        )

        gdf_centroids = gdf_centroids.drop(close_points.index)

    return final_centroids

school = remove_close_centroids(school)

school.explore()

# Оставляем только геометрию
school_geometry = school[['geometry']]

# Обновляем данные в модели города
city.update_services('school', school_geometry)

services_gdf = city.get_services_gdf()
services_gdf.head()


/usr/local/lib/python3.11/dist-packages/geopandas/array.py:1470: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 / UTM zone 38N (the single non-null crs provided).
  return GeometryArray(data, crs=_get_common_crs(to_concat))
/usr/local/lib/python3.11/dist-packages/geopandas/array.py:1470: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 / UTM zone 38N (the single non-null crs provided).
  return GeometryArray(data, crs=_get_common_crs(to_concat))
/usr/local/lib/python3.11/dist-packages/geopandas/array.py:1470: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 / UTM zone 38N (the single non-null crs provided).
  return GeometryArray(data, crs=_get_common_crs(to_concat))
/usr/local/lib/python3.11/dist-packages/geopandas/array.py:1470: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 / UTM zone 38N (the single n

,geometry,block_id,building_id,service_type,capacity,area,is_integrated
0,POINT (326597.822 6370355.923),18,734.0,hospital,110,3500.0,False
1,POINT (327467.669 6371926.149),20,NaN,school,250,3200.0,False
2,POINT (325344.880 6373907.859),44,8453.0,school,250,3200.0,False
3,POINT (327729.864 6371189.373),76,1087.0,school,250,3200.0,False
4,POINT (327375.341 6370724.167),80,2410.0,school,250,3200.0,False


KINDERGARTEN

In [ ]:
import geopandas as gpd
import pandas as pd

# Шаг 0: Загрузка kindergarten вручную из файла (например, 'my_kindergartens.geojson')
# Замените путь и формат файла на нужный вам
kindergarten = gpd.read_file("Kin_kindergarten_2010_centroids.geojson")  # или .shp, .gpkg, .csv и т.п.

# Приведение CRS к локальной системе координат
kindergarten = kindergarten.to_crs(local_crs)

# Шаг 1: Удаляем вложенные полигоны
def remove_inner_polygons(gdf_polygons):
    to_remove = []
    for idx, poly1 in gdf_polygons.iterrows():
        for idx2, poly2 in gdf_polygons.iterrows():
            if idx != idx2 and poly1.geometry.contains(poly2.geometry):
                to_remove.append(idx2)
    return gdf_polygons.drop(to_remove)

kindergarten = remove_inner_polygons(kindergarten)

# Шаг 2: Преобразуем полигоны в центроиды
kindergarten['geometry'] = kindergarten.centroid

# Шаг 3: Удаляем близкие центроиды, оставляя одну точку на буфер
def remove_close_centroids(gdf_centroids, buffer_distance=60):
    final_centroids = gpd.GeoDataFrame(columns=gdf_centroids.columns, crs=gdf_centroids.crs)

    while not gdf_centroids.empty:
        current_point = gdf_centroids.iloc[0]
        buffer = current_point.geometry.buffer(buffer_distance)
        close_points = gdf_centroids[gdf_centroids.geometry.within(buffer)]

        final_centroids = gpd.GeoDataFrame(
            pd.concat([final_centroids, gpd.GeoDataFrame([current_point])], ignore_index=True),
            crs=gdf_centroids.crs
        )

        gdf_centroids = gdf_centroids.drop(close_points.index)

    return final_centroids

kindergarten = remove_close_centroids(kindergarten)

kindergarten.explore()

# Оставляем только геометрию
kindergarten_geometry = kindergarten[['geometry']]

# Обновляем данные в модели города
city.update_services('kindergarten', kindergarten_geometry)

services_gdf = city.get_services_gdf()
services_gdf.head()


/usr/local/lib/python3.11/dist-packages/geopandas/array.py:1470: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 / UTM zone 38N (the single non-null crs provided).
  return GeometryArray(data, crs=_get_common_crs(to_concat))
/usr/local/lib/python3.11/dist-packages/geopandas/array.py:1470: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 / UTM zone 38N (the single non-null crs provided).
  return GeometryArray(data, crs=_get_common_crs(to_concat))
/usr/local/lib/python3.11/dist-packages/geopandas/array.py:1470: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 / UTM zone 38N (the single non-null crs provided).
  return GeometryArray(data, crs=_get_common_crs(to_concat))
/usr/local/lib/python3.11/dist-packages/geopandas/array.py:1470: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 / UTM zone 38N (the single n

,geometry,block_id,building_id,service_type,capacity,area,is_integrated
0,POINT (333155.239 6369476.188),4,9153.0,kindergarten,80,180.0,True
1,POINT (331399.806 6369719.899),6,1948.0,kindergarten,80,230.0,False
2,POINT (326597.822 6370355.923),18,734.0,hospital,110,3500.0,False
3,POINT (327467.669 6371926.149),20,NaN,school,250,3200.0,False
4,POINT (327418.506 6372004.238),20,11386.0,kindergarten,80,230.0,False


POLYCLINIC

In [ ]:
import geopandas as gpd
import pandas as pd

# Шаг 0: Загрузка polyclinic вручную из файла (например, 'my_polyclinics.geojson')
# Замените путь и формат файла на ваш собственный
polyclinic = gpd.read_file("Kin_polyclinic.geojson")  # можно .shp, .gpkg, .csv и т.д.

# Приведение CRS к локальной системе координат
polyclinic = polyclinic.to_crs(local_crs)

# Шаг 1: Удаляем вложенные полигоны
def remove_inner_polygons(gdf_polygons):
    to_remove = []
    for idx, poly1 in gdf_polygons.iterrows():
        for idx2, poly2 in gdf_polygons.iterrows():
            if idx != idx2 and poly1.geometry.contains(poly2.geometry):
                to_remove.append(idx2)
    return gdf_polygons.drop(to_remove)

polyclinic = remove_inner_polygons(polyclinic)

# Шаг 2: Преобразуем полигоны в центроиды
polyclinic['geometry'] = polyclinic.centroid

# Шаг 3: Удаляем близкие центроиды
def remove_close_centroids(gdf_centroids, buffer_distance=60):
    final_centroids = gpd.GeoDataFrame(columns=gdf_centroids.columns, crs=gdf_centroids.crs)

    while not gdf_centroids.empty:
        current_point = gdf_centroids.iloc[0]
        buffer = current_point.geometry.buffer(buffer_distance)
        close_points = gdf_centroids[gdf_centroids.geometry.within(buffer)]

        final_centroids = gpd.GeoDataFrame(
            pd.concat([final_centroids, gpd.GeoDataFrame([current_point])], ignore_index=True),
            crs=gdf_centroids.crs
        )

        gdf_centroids = gdf_centroids.drop(close_points.index)

    return final_centroids

polyclinic = remove_close_centroids(polyclinic)

polyclinic.explore()

# Оставляем только геометрию
polyclinic_geometry = polyclinic[['geometry']]

# Обновляем модель города
city.update_services('polyclinic', polyclinic_geometry)

services_gdf = city.get_services_gdf()
services_gdf.head()


/usr/local/lib/python3.11/dist-packages/geopandas/array.py:1470: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 / UTM zone 38N (the single non-null crs provided).
  return GeometryArray(data, crs=_get_common_crs(to_concat))
/usr/local/lib/python3.11/dist-packages/geopandas/array.py:1470: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 / UTM zone 38N (the single non-null crs provided).
  return GeometryArray(data, crs=_get_common_crs(to_concat))
/usr/local/lib/python3.11/dist-packages/geopandas/array.py:1470: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 / UTM zone 38N (the single non-null crs provided).
  return GeometryArray(data, crs=_get_common_crs(to_concat))
/usr/local/lib/python3.11/dist-packages/geopandas/array.py:1470: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 / UTM zone 38N (the single n

,geometry,block_id,building_id,service_type,capacity,area,is_integrated
0,POINT (333155.239 6369476.188),4,9153.0,kindergarten,80,180.0,True
1,POINT (331399.806 6369719.899),6,1948.0,kindergarten,80,230.0,False
2,POINT (326255.780 6370849.213),12,81.0,polyclinic,100,850.0,False
3,POINT (326597.822 6370355.923),18,734.0,hospital,110,3500.0,False
4,POINT (327467.669 6371926.149),20,NaN,school,250,3200.0,False
